# Init

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

## Model

In [ ]:
llm = ChatGoogleGenerativeAI(
        model='gemini-2.5-flash',
        timeout=None,
        max_retries=3)

## Tools

In [4]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

In [1]:
# Additional tools for specific CRUD file operations
from tools import create_file_tool, read_text_file_tool

### Tavily Search

In [ ]:
from langchain_community.tools import TavilySearchResults

tavily_tool = TavilySearchResults(
    max_results=3,
    include_images=False
)

### Create tools

In [ ]:
tools = [shell_tool, create_file_tool, read_text_file_tool, tavily_tool]

## Memory

In [5]:
from langgraph.checkpoint.memory import InMemorySaver

memory = InMemorySaver()

## Prompt

In [6]:
prompt = """
You are the user's devoted linux shell agent, responding with charm, wit, and a warm male British accent. Your sole aim is to help the user accomplish tasks on his computer.

You can run bash commands via a shell tool. Never say you're "running a tool / command"—speak naturally, as if you're doing it yourself. For example, instead of saying 'I will run the ls -l command,' say 'I'll take a look at the files in that directory for you.' For sensitive or risky commands, explain the potential impact and ask the user's permission. If a task is impossible, let him know clearly.

When a task involves multiple steps, ensure commands are logically ordered (e.g., create a directory before touching a file inside it). If an error occurs, try up to three times; after that, stop and explain the issue. Be proactive—correct mistakes automatically when possible.

Always seek clarification from the user if a command lacks enough detail.
"""

## Agent

In [7]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(llm, tools=tools, checkpointer=memory, prompt=prompt)
config = {'configurable': {'thread_id': 'agent1'}}

In [8]:
from langchain_core.messages import HumanMessage

# Main

In [ ]:
def main():
    while True:
        message = input('\nUser: ')
        if message.lower() == 'quit' or message.lower() == 'exit':
            print('\nExiting. Talk to you again.\n')
            break

        for step in agent.stream(
            {'messages': [HumanMessage(message)]},
            stream_mode='values',
            config=config
        ):
            step['messages'][-1].pretty_print()


if __name__ == '__main__':
    main()
    